In [1]:
import fiona
import rasterio
import rasterio.mask
import numpy as np
import math

In [2]:
from skimage import io, transform

In [3]:
def img2Vec(imgPath, listVar):
    imgSize = 45
    img = listVar[0]
    img = io.imread(imgPath, as_grey = True, plugin = "gdal")
    img = transform.resize(img, (imgSize, math.ceil(img.shape[1]/img.shape[0]*imgSize)))
    img.resize(1,img.shape[0]*img.shape[1])
    return img.copy()

In [4]:
imgPaths = []
imgDir = '/home/lamductan/Hydroviet/Tonlesap/RGB/'
yearLabel = '2017'
for i in range(1,13):
    imgPaths.append(imgDir + 'Tonlesap_' + yearLabel + str(i).zfill(2) + '.tif')

In [5]:
Y = np.array([[4.4, 1.7, 2, 3, 1.8, 2.2, 4, 6, 7.8, 8.2, 7.6, 6.1]])

In [6]:
def prepairData(imgPaths):
    listVar = [1]
    X = []
    for path in imgPaths:
        X.append(img2Vec(path, listVar))
    return np.array(X)

In [7]:
X = prepairData(imgPaths)

/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:49: UserWarning: `as_grey` has been deprecated in favor of `as_gray`
  warn('`as_grey` has been deprecated in favor of `as_gray`')
/usr/local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [8]:
X = X.reshape(X.shape[0], X.shape[2])

In [9]:
X = X/np.max(X)

In [10]:
X = X.T
a = np.ones((1,X.shape[1]))
X = np.concatenate((a,X), axis=0)

In [11]:
testPercent = 3/4

X_train = X[:,:int(testPercent*X.shape[1])]
Y_train = Y[:,:int(testPercent*Y.shape[1])]

print(X_train)
print(Y_train)

X_test = X[:,math.ceil(testPercent*X.shape[1]):]
Y_test = Y[:,math.ceil(testPercent*X.shape[1]):]

[[1. 1. 1. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[4.4 1.7 2.  3.  1.8 2.2 4.  6.  7.8]]


In [12]:
def costFunction(theta, X, Y):
    m = X.shape[1]
    A = theta.dot(X) - Y
    return 1/(2*m)*A.dot(A.T)

In [13]:
def gradientDescent(theta, X, Y):
    m = X.shape[1]
    matMul = theta.dot(X)-Y
    grad = np.zeros(theta.shape)
    grad[0,0] = 1/m * np.sum(matMul)
    for i in range(1, theta.shape[1]):
        grad[0,i] = 1/m * matMul.dot(X[i].T)
    return grad

In [14]:
def linearRegressionWithGradientDescent(X_train, Y_train, alpha = 0.001):
    #iters = 100
    iters = 0
    theta = np.random.random((1,X_train.shape[0]))
    cost = 10000
    threshold = 0.01
    while cost > threshold and iters < 100000:
        cost = costFunction(theta, X_train, Y_train)
        if iters % 100 == 0:
            print(cost)
        grad = gradientDescent(theta, X_train, Y_train)
        theta -= alpha*grad
        #iters -= 1
        iters += 1
    print('iters = %d' % iters)
    return theta

In [15]:
theta = linearRegressionWithGradientDescent(X_train, Y_train)

[[2066.66562685]]
[[2.04292718]]
[[0.9941997]]
[[0.77507473]]
[[0.6133482]]
[[0.49345365]]
[[0.40420715]]
[[0.33742917]]
[[0.28713683]]
[[0.24895339]]
[[0.21967655]]
[[0.19696274]]
[[0.17909637]]
[[0.16482107]]
[[0.15321634]]
[[0.14360731]]
[[0.1354988]]
[[0.12852709]]
[[0.12242464]]
[[0.11699432]]
[[0.11209054]]
[[0.10760547]]
[[0.10345893]]
[[0.09959105]]
[[0.09595682]]
[[0.09252219]]
[[0.08926114]]
[[0.0861536]]
[[0.08318388]]
[[0.08033952]]
[[0.07761051]]
[[0.07498861]]
[[0.07246695]]
[[0.0700397]]
[[0.06770181]]
[[0.06544881]]
[[0.06327674]]
[[0.06118199]]
[[0.05916126]]
[[0.05721151]]
[[0.05532989]]
[[0.05351373]]
[[0.05176051]]
[[0.05006784]]
[[0.04843346]]
[[0.0468552]]
[[0.045331]]
[[0.04385887]]
[[0.04243694]]
[[0.04106339]]
[[0.03973647]]
[[0.03845452]]
[[0.03721592]]
[[0.03601913]]
[[0.03486267]]
[[0.03374511]]
[[0.03266507]]
[[0.03162123]]
[[0.03061233]]
[[0.02963713]]
[[0.02869445]]
[[0.02778316]]
[[0.02690216]]
[[0.02605039]]
[[0.02522686]]
[[0.02443056]]
[[0.02366057]]


In [16]:
def predict(theta, X):
    return theta.dot(X.T)

In [17]:
train = []
for i in range(X_train.shape[1]):
    train.append(predict(theta, X_train[:,i]))

print(np.array(train))

[[4.09966843]
 [1.86658081]
 [2.00725151]
 [3.13894159]
 [1.6576701 ]
 [2.22518639]
 [4.02333316]
 [6.12472712]
 [7.72489047]]


In [18]:
print(Y_train.T)

[[4.4]
 [1.7]
 [2. ]
 [3. ]
 [1.8]
 [2.2]
 [4. ]
 [6. ]
 [7.8]]


In [19]:
pred = []
for i in range(X_test.shape[1]):
    pred.append(predict(theta, X_test[:,i]))

print(np.array(pred))

[[9.25500874]
 [7.68070611]
 [5.11095436]]


In [20]:
print(Y_test.T)

[[8.2]
 [7.6]
 [6.1]]
